In [ ]:
import os
import json
from pathlib import Path
import pandas as pd

In [ ]:
INPUT_DIR = Path('input')
OUTPUT_DIR = Path('output')
estates = [x.name for x in INPUT_DIR.glob('*') if x.is_dir()]
estates

In [ ]:
# @dev: this will only have to be done once

# go through each estate/image directory and rename the 
# images by extracting the number from the filename.
# The format current is 1.asdfasdf.png, 2.asdfasdf.png, etc. 
# We want to rename them to 1.png, 2.png, etc.
# for estate in estates:
#     images = [x.name for x in Path(f'inputs/{estate}/images').glob('*.png')]
#     print(images)
#     for image in images:
#         os.rename(f'inputs/{estate}/images/{image}',
#                    f'inputs/{estate}/images/{image.split(".")[0]}.png')

In [ ]:
ipfs_image_base_url = "https://ipfs.fanfire.ai/ipfs/QmV7WGPHwSYYHcM6pUocKmVTi3BZBZVi61tkAoF8iGNS5e"

In [ ]:
bottels = {}
for estate in estates:
    df = pd.read_csv(INPUT_DIR / estate / 'bottles.csv')
    
    print("======")
    print(f"{estate}: {len(df)}")
    df["Recorked"] = df["Recorked"].apply(lambda x: "Yes" if "yes" in x.lower() else "No")
    df['Wine Name'] = df['Estate'] + ' ' + df['Name'] + ' ' + df['Vintage'].astype(str)
    df['Volume [ml]'] = df['Volume [ml]'].apply(lambda x: str(x).replace("ml", "").strip()).astype(int)
    df["Wine Description (Highlights of vintage)"].fillna("", inplace=True)
    df["Rating 1"].fillna("", inplace=True)
    df["Rating 2"].fillna("", inplace=True)
    df["Rating 3"].fillna("", inplace=True)

    df["Wine Name"] = df["Wine Name"].str.replace('"', "'").replace('\n', '')
    df["Wine Description (Highlights of vintage)"] = df["Wine Description (Highlights of vintage)"].str.replace('"', "'").replace('\n', '')
    df["Rating 1"] = df["Rating 1"].str.replace('"', "'").replace('\n', '')
    df["Rating 2"] = df["Rating 2"].str.replace('"', "'").replace('\n', '')
    df["Rating 3"] = df["Rating 3"].str.replace('"', "'").replace('\n', '')
    df["Estate"] = df["Estate"].str.replace('"', "'").replace('\n', '')
    df["Name"] = df["Name"].str.replace('"', "'").replace('\n', '')
    
    bottels[estate] = df

In [ ]:
DEFAULT_JSON = """{
    "name": "NAME",
    "external_url": "https://strauss.fanfire.ai",
    "description": "DESCRIPTION",
    "image":  "IMAGE_URL",
    "attributes": [
      {
        "trait_type": "Volume (ml)",
        "value": VOLUME
      },
      {
        "trait_type": "Vintage",
        "value": "VINTAGE"
      },
      {
        "trait_type": "Recorked",
        "value": "RECORKED"
      },
      {
        "trait_type": "Authentication Code",
        "value": "AUTH_CODE"
      },
       {
        "trait_type": "Rating 1",
        "value": "RATING_1"
      },
      {
        "trait_type": "Rating 2",
        "value": "RATING_2"
      },
      {
        "trait_type": "Rating 3",
        "value": "RATING_3"
      }
    ]
}
  """

DEFAULT_BASKET_JSON = """{
    "name": "NAME",
    "external_url": "https://strauss.fanfire.ai",
    "description": "DESCRIPTION",
    "image":  "IMAGE_URL",
    "attributes": [
      {
        "trait_type": "Nr Bottles",
        "value": NR_BOTTLES
      },
      {
        "trait_type": "Basket Id",
        "value": BASKET_ID
      }
    ]
}
  """

In [ ]:
DEFAULT_COLLECTION = """
{
    "image": "COLLECTION_IMAGE_URL",
    "description": "COLLECTION_DESCRIPTION",
    "name": "COLLECTION_NAME",
    "seller_fee_basis_points": "800",
    "fee_recipient": "0x25375E1DaFa37a31069d323a37A6f93eaF356123"
}
"""

In [ ]:
collection_descriptions = {
    "Yquem": "Château d`Yquem, often described as the greatest sweet wine in the world and the only Sauternes First Growth of Bordeaux. Few wines in the world have produced consistently noble quality for more than 4 centuries.\\n\\nEstablished in 1593, the property consists of a 110-hectare vineyard planted with 80% Semillon and 20% Sauvignon Blanc. The wines are created from botrytized fruit that is picked by hand in several passes 'tiers' by up to 150 harvesters. Château d'Yquem is the product of meticulous work in the vineyard, along with the estate's exceptional terroir, and centuries of inherited knowledge since 1593, it has gained the reputation of the world's finest sweet wine.\\n\\nIn the Bordeaux Classification of 1855, Château d'Yquem was given the unique and unsurpassable rank of Premier Cru Supérieur, ahead of all other Sauternes, indicating its perceived superiority and ability to command by far the highest prices. The wines of Château d'Yquem are characterised by their complexity, concentration and longevity. In a good year, a bottle will only begin to show its qualities after a decade or two of cellaring and with proper care, will keep for a century or more, gradually adding layers of complexity.",
    "Penfolds": "Penfolds was founded by Dr Christopher and Mary Penfold in 1844, who arrived in Australia with their vine cuttings in hand. They had no prior history in winemaking but set about inventing tonics, brandies, and fortified wines. Mary took charge, blending the wines to her taste and enforcing the winemaking philosophies that Penfolds still follows today. Penfolds Grange was created in the 1950s and holds a high reputation worldwide.\\n\\nSublime, voluptuous, beguiling, inviting, opulent, layered. These are some of the words often used to describe Penfolds Grange. Unlike the most expensive wines of Europe, Grange is not a single vineyard production, but rather a carefully blended composition of high-quality grapes harvested from a wide region. The craft and expertise that goes into each vintage gives Penfolds Grange its prestige pedigree.",
    "Harlan": "Harlan Estate, seen as Napa's 'First-Growth' is a high-profile, critically acclaimed wine producer in Oakville AVA, Napa Valley, situated on the hills just north of Napa town. Its Cabernet Sauvignon-based flagship wine is one of California's most sought-after wines, regularly commanding exuberant prices.\\n\\nOf the so-called 'cult' wineries that burst on the scene in the 1990s, Harlan is most fascinating, where every detail, from vineyards to label to winery design, reflects the relentless drive for quality. The wines are bold, voluptuous, and rich, yet polished and balanced, despite their alcohol.\\n\\nThe debut 1990 Harlan Estate, of which only 300 cases were made, were all sold directly to the mailing list. Over the years, they've attained global success. Demand outstrips supply, even with a hefty pre-release bottle price.\\n\\nThe 19th-century image that adorns the label came from the 250-year-old American Bank Note Company and it's printed on banknote paper, way ahead on the anti-counterfeit curve. The idea was to create a label that looked like an engraving for a bottle, that by itself was a piece of art.",
    "Romanee": "Domaine de la Romanée-Conti is one of the most enigmatic names in the wine world, conjuring up notions of exclusivity and desirability. It is the only Burgundy wine producer with a status comparable to the top Bordeaux producers. Its prestige, and historical links with royalty, has made it the foremost name in Burgundy, a region which has been experiencing significantly increased demand over recent years, becoming a priority for international collectors.\\n\\nOne of the most renowned vineyards in the world, located in the commune of Vosne-Romanée. The wines of incredible Grand cru sites of Échézeaux, Grands Échézeaux, La Tâche, Richebourg and Romanée-Saint-Vivant, are praised for their subtle and mysterious nature.\\n\\nGuided by the relentless pursuit of excellence, passed down from generation to generation, Domaine de la Romanée-Conti's mission is to make its Grands Crus the greatest in all of Burgundy. Few would dispute that the wines of DRC are some of the world's finest.",
    "Screaming-Eagle": "The highly coveted California Cabernet is one of the most collectible, sought-after, wines of the world, due to its high critic ratings and extremely limited production, 'Cult' status was quickly sealed. Notoriously difficult to secure a bottle due to the small production and exclusive DTC mailing list (with an incredibly long waiting list of more than a decade). Its maiden vintage production of only 225 cases available to purchase, saw prices skyrocket and demand increased rapidly.\\n\\nScreaming Eagle is one of California's top investment wines, which has seen sustained growth over the past decade, with a rising demand. As an investment or as a wine to simply take pleasure in, Screaming Eagle never fails to impress.\\n\\nScreaming Eagle has been recognised by the greatest international wine critics for its extreme finesse since the release of its first vintage in 1992. The highly influential US wine critic, Robert Parker, scored the 1992 maiden vintage a whopping 99 out of 100 points, igniting overnight success for the young winery. Since then, the 1997 and 2007 vintages both achieved the coveted 100-point rating from Parker, the Napa Cabernet aficionado, held responsible for Screaming Eagle's quick rise to fame! 2010 Sauvignon Blanc is the first vintage of the estate's extremely rare Sauvignon Blanc. There are only two acres of this white wine grape on the Screaming Eagle estate — they made just 200 bottles in 2010!",
    "Mouton": "Château Mouton Rothschild is not only one of the finest Bordeaux wines, but it is also one of the world's greatest. The First Growth from Pauillac holds an incredible track record for the most consistent and long-lived wines in all of Bordeaux. It can be argued that the modern age of Château Mouton Rothschild arose along with the modernization of the entire Bordeaux wine region.\\n\\nEach year since 1945, the Château Mouton Rothschild label has been illustrated with an artwork by a leading painter, specially created for the vintage. Thus, the most famous names in contemporary painting have been brought together in a collection to which a new work is added each year. From one label to another, Mouton Rothschild has thus brought together some of the most celebrated artists of their day, including Miró, Chagall, Braque, Picasso, Tàpies, Francis Bacon, Dali, Balthus, Jeff Koons and even Prince Charles, the Prince of Wales. The artists have complete freedom of creation, though certain themes, such as the vine, the pleasure of drinking and the ram, the Mouton Rothschild emblem, have proved a particularly rich source of inspiration.",
}

collection_names = {
    "Yquem": "Château d'Yquem 1892-2010",
    "Penfolds": "Penfolds Grange 1951-2008",
    "Harlan": "Harlan Estate 1990-2012",
    "Romanee": "Domaine de la Romanée-Conti 1999-2015",
    "Screaming-Eagle": "Screaming Eagle 1995-2010",
    "Mouton": "Château Mouton Rothschild 1945-2014",
}


collection_counts = {
    "Yquem": 184,
    "Penfolds": 232,
    "Harlan": 40,
    "Romanee": 80,
    "Screaming-Eagle": 19,
    "Mouton": 176,
}

collection_basket_ids = {
    "Penfolds": 0,
    "Harlan": 1,
    "Mouton": 2,
    "Romanee": 3,
    "Screaming-Eagle": 4,
    "Yquem": 5,
}

basket_description_footer = "\\n\\nThe code for this project can be found on [GitHub](https://github.com/fan-fire/strauss-and-co). Note that the tokens in this collection serve as a [basket](https://github.com/fan-fire/NFB) where each basket owns all the underlying tokens for each estate, and only the owner of the basket can open and remove wines from the basket. Reach out to us on our [website](https://www.fanfire.ai/) or [Discord](https://discord.gg/kB7GFaBX7E) if you have any questions."

basket_description = """Introducing Six Coveted Collections of Fine Wines\\n\\nStep into the world of exclusive wine ownership with Strauss & Co, where your prized collection of fine wines is backed by cutting-edge digital ownership technology. With our first-of-its-kind digital provenance certificates, your wine collection becomes an exceptional blend of personal enjoyment and investment potential.\\n\\nEach vertical collection has been meticulously stored under optimal conditions by Strauss & Co, ensuring the preservation of their exquisite flavors until the moment you decide to redeem an individual bottle or the entire collection. As the owner of the digital ownership certificate ('NFT'), you have the flexibility to hold onto it or trade parts or the entirety of your collection as you please.\\n\\nThe NFT serves as your key to accessing and redeeming the wines within your collection. It is crucial that you retain access to this unique certificate, as only the current NFT holder can claim the wine. To assist you throughout your ownership journey, Fanfire is ready to provide comprehensive technical support and guidance. Our team is committed to empowering you with control over your ownership, irrespective of your prior knowledge of crypto, blockchain, and NFTs.\\n\\nExplore our six esteemed collections, each representing a prestigious estate:\\n\\n### [Penfolds Grange 1951-2008](https://opensea.io/collection/0d246974176dc089d131040166c8ef617ccad578)\\nEmbark on a journey through time with Penfolds Grange, showcasing an exquisite range of vintages from 1951 to 2008. Experience the profound flavors and impeccable craftsmanship of Australia's most celebrated wine label.\\n\\n### [Harlan Estate 1990-2012](https://opensea.io/collection/a44ca542eec0f8f91d2cbc63230b1b9f9816f9f9)\\nUncover the allure of Harlan Estate, encompassing limited vintages from 1990 to 2012. Imbibe the exceptional wines hailing from the iconic Napa Valley vineyard, renowned for their depth, complexity, and elegance.\\n\\n### [Château Mouton Rothschild 1945-2014](https://opensea.io/collection/038c0998f580798f849ee8841a7e5dd5040f8df8)\\nEmbark on a journey through the historic legacy of Château Mouton Rothschild, spanning vintages from 1945 to 2014. Delight in the iconic wines produced by this illustrious Bordeaux estate, celebrated for their opulence and age-worthiness.\\n\\n### [Domaine de la Romanée-Conti 1999-2015](https://opensea.io/collection/ede4662aeab7fb9ac316476643cc836a171a12b0)\\nDiscover the epitome of Pinot Noir excellence with Domaine de la Romanée-Conti. This collection brings together exceptional vintages from 1999 to 2015, showcasing the timeless elegance and unmatched finesse of Burgundy's renowned vineyard.\\n\\n### [Screaming Eagle 1995-2010](https://opensea.io/collection/0f580daf909e1dc0892301222ed60f59ee6f3770)\\nImmerse yourself in the allure of Screaming Eagle, an extraordinary collection featuring limited vintages from 1995 to 2010. Savor the highly sought-after wines from this cult Napa Valley winery, known for their unparalleled quality and rarity.\\n\\n### [Château d'Yquem 1892-2010](https://opensea.io/collection/6058ee21b093ee36333aee29996ae9f967bc52eb)\\nImmerse yourself in the rich heritage of Château d'Yquem, spanning vintages from 1892 to 2010. Indulge in the golden nectar produced by this iconic French estate known for its legendary sweet white wines.\\n\\nEmbrace the privilege of owning these exceptional wine collections, empowered by digital ownership and the potential for unparalleled enjoyment and investment. With Strauss & Co and Fanfire by your side, we are committed to ensuring your ownership experience is seamless and rewarding.\\n\\nPlease note that the collection links above direct you to OpenSea, a popular marketplace for NFTs (Non-Fungible Tokens), where you can explore and view the tokens within each of these collections."""

In [ ]:
estate_tokens = {}
estate_collections = {}
for estate, df in bottels.items():
    estate_tokens[estate] = []
    collection_str = (
        DEFAULT_COLLECTION.replace(
            "COLLECTION_IMAGE_URL", ipfs_image_base_url + f"/{estate}/collection.jpeg"
        )
        .replace("COLLECTION_DESCRIPTION", collection_descriptions[estate])
        .replace("COLLECTION_NAME", f"{collection_names[estate]} Collection")
    )
    try:
        collection_dict = json.loads(collection_str)
    except:
        print(collection_str)
        raise
    estate_collections[estate] = collection_dict

    basket_str = (
        DEFAULT_BASKET_JSON.replace(
            "IMAGE_URL", ipfs_image_base_url + f"/{estate}/collection.jpeg",
        )
        .replace("DESCRIPTION", collection_descriptions[estate]+basket_description_footer)
        .replace("NAME", f"{collection_names[estate]} Collection")
        .replace("NR_BOTTLES", str(collection_counts[estate]))
        .replace("BASKET_ID", str(collection_basket_ids[estate]))
    )

    try:
        basket_dict = json.loads(basket_str)
    except:
        print(basket_str)
        raise

    basket_collection_str = (
        DEFAULT_COLLECTION.replace(
            "COLLECTION_IMAGE_URL",
            "https://ipfs.fanfire.ai/ipfs/QmV7WGPHwSYYHcM6pUocKmVTi3BZBZVi61tkAoF8iGNS5e/2023-auction.jpeg",
        ).replace("COLLECTION_DESCRIPTION", basket_description+basket_description_footer)
        .replace("COLLECTION_NAME", "Strauss & Co Rare Wine Collections 2023")
    )

    try:
        basket_collection_dict = json.loads(basket_collection_str)
    except:
        print(basket_collection_str)
        raise

    basket_dir = OUTPUT_DIR / 'baskets'
    os.makedirs(basket_dir, exist_ok=True)
    with open(basket_dir / f"{collection_basket_ids[estate]}.json", "w") as f:
        json.dump(basket_dict, f, indent=2)
    
    with open(basket_dir / f"collection.json", "w") as f:
        json.dump(basket_collection_dict, f, indent=2)


    for tokenId, bottel in df.iterrows():
        json_str = (
            DEFAULT_JSON.replace("NAME", bottel["Wine Name"])
            .replace("DESCRIPTION", bottel["Wine Description (Highlights of vintage)"])
            .replace(
                "IMAGE_URL",
                ipfs_image_base_url
                + f"/{estate}/images/{bottel['Image name & no.']}.png",
            )
            .replace("VOLUME", str(bottel["Volume [ml]"]))
            .replace("VINTAGE", str(bottel["Vintage"]))
            .replace("RECORKED", bottel["Recorked"])
            .replace("AUTH_CODE", str(bottel["Authentication code"]))
            .replace("RATING_1", str(bottel["Rating 1"]))
            .replace("RATING_2", str(bottel["Rating 2"]))
            .replace("RATING_3", str(bottel["Rating 3"]))
        )
        try:
            json_dict = json.loads(json_str)
        except:
            print(json_str)
            raise
        estate_tokens[estate].append(json_dict)
    print(f"Done with {estate}")

In [ ]:
deployed_addresses = {
    "Screaming-Eagle": "0x0F580daF909e1Dc0892301222eD60F59eE6f3770",
    "Yquem": "0x6058ee21b093EE36333Aee29996aE9f967bc52Eb",
    "Penfolds": "0x0d246974176dc089d131040166c8Ef617CcAd578",
    "Harlan": "0xa44Ca542EEc0f8F91D2cBc63230B1b9f9816F9F9",
    "Romanee": "0xedE4662aEAB7FB9aC316476643cc836A171A12B0",
    "Mouton": "0x038C0998f580798F849Ee8841a7E5dD5040F8dF8",
}

In [ ]:
for estate, tokens in estate_tokens.items():
    assert len(tokens) == collection_counts[estate], f"{estate} has {len(tokens)} tokens, but should have {collection_counts[estate]}"
    os.makedirs(OUTPUT_DIR / estate, exist_ok=True)
    for token_id, token in enumerate(tokens):
        filename = f"{estate}/{token_id}.json"
        with open(OUTPUT_DIR / filename, "w") as f:
            json.dump(token, f, indent=4)
        token["tokenId"] = token_id
    with open(OUTPUT_DIR / estate / "collection.json", "w") as f:
        json.dump(estate_collections[estate], f, indent=4)

    df_tokens = (
        pd.DataFrame(tokens)
        .rename(
            columns={
                "name": "Name",
                "description": "Description",
                "image": "Image",
            }
        )
        .drop(columns=["external_url"])
    )

    def get_attribute(x, att_name):
        for att in x:
            if att["trait_type"] == att_name:
                return att["value"]
        return None

    df_tokens["Volume (ml)"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Volume (ml)")
    )
    df_tokens["Vintage"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Vintage")
    )
    df_tokens["Recorked"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Recorked")
    )
    df_tokens["Authentication Code"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Authentication Code")
    )
    df_tokens["Rating 1"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Rating 1")
    )
    df_tokens["Rating 2"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Rating 2")
    )
    df_tokens["Rating 3"] = df_tokens["attributes"].apply(
        lambda x: get_attribute(x, "Rating 3")
    )
    df_tokens["URL"] = df_tokens["tokenId"].apply(
        lambda x: f"https://opensea.io/assets/matic/{deployed_addresses[estate]}/{x}"
    )
    df_tokens.drop(columns=["attributes"], inplace=True)
    df_tokens.rename(columns={"tokenId": "Token ID"}, inplace=True)
    df_tokens.to_csv(OUTPUT_DIR / f"{estate}.csv", index=False)